In [ ]:
from scipy import stats

In [17]:
def get_valutation_from_txt(path):
    andrea_list = []
    fabio_list = []
    with open(path, 'r') as file:
        for line in file.readlines():
            andrea_list.append(float(line.split('    ')[2]))
            fabio_list.append(float(line.split('    ')[3]))

    return andrea_list, fabio_list

In [18]:
PATH = '/home/fazza/Documents/TLN-LAB-PART-2/resources/annotations_fabio_andrea.txt'
a,f = get_valutation_from_txt(PATH)

#print(f"Lista di Andrea: {a}")
#print(f"Lista di Fabio: {f}")

print("\nSPEARMAN'S CORRELATION COEFFICIENT\n")
print(f"{stats.spearmanr(a, f).correlation}")

print("\nPEARSON'S CORRELATION COEFFICIENT\n")
print(f"{stats.pearsonr(a, f)}")


SPEARMAN'S CORRELATION COEFFICIENT

0.8969380835014799

PEARSON'S CORRELATION COEFFICIENT

(0.9316329502979276, 9.705442539988869e-23)
